<a href="https://colab.research.google.com/github/UKJaagadhep/Data-science-and-machine-learning/blob/main/YOLO/YOLO_FROM_SCRATH_SKELETON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **IMPORTING NECESSARY LIBRARIES**

In [ ]:
import xml.etree.ElementTree as ET
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import (GlobalAveragePooling2D, Activation, MaxPooling2D, Add, Conv2D, MaxPool2D, Dense,
                                     Flatten, InputLayer, BatchNormalization, Input, Embedding, Permute,
                                     Dropout, RandomFlip, RandomRotation, LayerNormalization, MultiHeadAttention,
                                     RandomContrast, Rescaling, Resizing, Reshape, LeakyReLU)
from tensorflow.keras.losses import BinaryCrossentropy,CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy,TopKCategoricalAccuracy, CategoricalAccuracy, SparseCategoricalAccuracy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (Callback, CSVLogger, EarlyStopping, LearningRateScheduler,
                                        ModelCheckpoint, ReduceLROnPlateau)
from tensorflow.keras.regularizers import L2, L1
from tensorflow.keras.initializers import RandomNormal
import albumentations as A
import shutil
import cv2
import time
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pathlib
import os

# **DOWNLOADING DATASET FROM KAGGLE**

In [ ]:
!pip install -q kaggle
!mkdir ~/.kaggle
!cp kaggle.json  ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d huanghanchina/pascal-voc-2012

mkdir: cannot create directory ‘/root/.kaggle’: File exists
100% 3.62G/3.63G [00:53<00:00, 83.6MB/s]
100% 3.63G/3.63G [00:53<00:00, 72.9MB/s]


In [ ]:
!unzip "/content/pascal-voc-2012.zip" -d "/content/dataset"

Streaming output truncated to the last 5000 lines.
  inflating: /content/dataset/voc2012/VOC2012/SegmentationClass/2008_001829.png  
  inflating: /content/dataset/voc2012/VOC2012/SegmentationClass/2008_001874.png  
  inflating: /content/dataset/voc2012/VOC2012/SegmentationClass/2008_001876.png  
  inflating: /content/dataset/voc2012/VOC2012/SegmentationClass/2008_001882.png  
  inflating: /content/dataset/voc2012/VOC2012/SegmentationClass/2008_001885.png  
  inflating: /content/dataset/voc2012/VOC2012/SegmentationClass/2008_001895.png  
  inflating: /content/dataset/voc2012/VOC2012/SegmentationClass/2008_001896.png  
  inflating: /content/dataset/voc2012/VOC2012/SegmentationClass/2008_001926.png  
  inflating: /content/dataset/voc2012/VOC2012/SegmentationClass/2008_001966.png  
  inflating: /content/dataset/voc2012/VOC2012/SegmentationClass/2008_001971.png  
  inflating: /content/dataset/voc2012/VOC2012/SegmentationClass/2008_001992.png  
  inflating: /content/dataset/voc2012/VOC2012/S

# **CREATING TRAIN AND VALIDATION DATASETS**

In [ ]:
val_list=['2007_000027.jpg','2007_000032.jpg','2007_000033.jpg','2007_000039.jpg','2007_000042.jpg','2007_000061.jpg',
          '2007_000063.jpg','2007_000068.jpg','2007_000121.jpg','2007_000123.jpg','2007_000129.jpg','2007_000170.jpg',
          '2007_000175.jpg','2007_000187.jpg','2007_000241.jpg','2007_000243.jpg','2007_000250.jpg','2007_000256.jpg',
          '2007_000272.jpg','2007_000323.jpg','2007_000332.jpg','2007_000333.jpg','2007_000346.jpg','2007_000363.jpg',
          '2007_000364.jpg','2007_000392.jpg','2007_000423.jpg','2007_000452.jpg','2007_000464.jpg','2007_000480.jpg',
          '2007_000491.jpg','2007_000504.jpg','2007_000515.jpg','2007_000528.jpg','2007_000529.jpg','2007_000549.jpg',
          '2007_000559.jpg','2007_000572.jpg','2007_000584.jpg','2007_000629.jpg','2007_000636.jpg','2007_000645.jpg',
          '2007_000648.jpg','2007_000661.jpg','2007_000663.jpg','2007_000664.jpg','2007_000676.jpg','2007_000713.jpg',
          '2007_000720.jpg','2007_000727.jpg','2007_000733.jpg','2007_000738.jpg','2007_000762.jpg','2007_000768.jpg',
          '2007_000783.jpg','2007_000793.jpg','2007_000799.jpg','2007_000804.jpg','2007_000807.jpg','2007_000822.jpg',
          '2007_001299.jpg','2007_001311.jpg','2007_001321.jpg','2007_001340.jpg']

In [ ]:
!mkdir /content/dataset/VOC2012/ValJPEGImages/
!mkdir /content/dataset/VOC2012/ValAnnotations/

In [ ]:
train_images = "/content/dataset/VOC2012/JPEGImages"
train_maps = "/content/dataset/VOC2012/Annotations"

val_images = "/content/dataset/VOC2012/ValJPEGImages"
val_maps = "/content/dataset/VOC2012/ValAnnotations"


In [ ]:
for name in val_list:
  shutil.move(train_maps+name[:-3]+"xml", val_maps+name[:-3]+"xml")
for name in val_list:
  shutil.move(train_images+name, val_images+name)

# **CLASSES AND CONFIGURATION**

In [ ]:
classes=['aeroplane','bicycle','bird','boat','bottle','bus','car','cat','chair','cow','diningtable',
         'dog','horse','motorbike','person','pottedplant','sheep','sofa','train','tvmonitor']

In [ ]:
b = 2
n_classes = len(classes)
obj_ind = n_classes + (5 * b)
h, w = 224, 224
split_size = h//32 #S : no. of grid cells in a row/column
n_epochs = 135
lr = 5e-4
batch_size = 32
n_filters = 512

# **TENSORFLOW DATASET CREATION AND PREPROCESSING**

In [ ]:
im_paths=[] #TO SAVE THE PATH OF EACH IMAGE
xml_paths=[] #TO SAVE THE PATH OF EACH CORRESPONDING .xml FILE

val_im_paths=[]
val_xml_paths=[]

for i in os.listdir(train_maps):
  im_paths.append(train_images+i[:-3]+'jpg')
  xml_paths.append(train_maps+i)

for i in os.listdir(val_maps):
  val_im_paths.append(val_images+i[:-3]+'jpg')
  val_xml_paths.append(val_maps+i)

print(len(im_paths),len(xml_paths))
print(len(val_im_paths),len(val_xml_paths))

In [ ]:
train_dataset=tf.data.Dataset.from_tensor_slices((im_paths,xml_paths))
val_dataset=tf.data.Dataset.from_tensor_slices((val_im_paths,val_xml_paths))

In [ ]:
for i in val_dataset.take(1):
  print(i)

In [ ]:
def preprocess_xml(filename):
  tree = ET.parse(filename)
  root = tree.getroot()
  size_tree = root.find('size')
  height = float(size_tree.find('height').text)
  width = float(size_tree.find('width').text)
  bounding_boxes=[]
  for object_tree in root.findall('object'): #WE COULD HAVE MULTIPLE OBJECTS IN A SINGLE IMAGE
    for bounding_box in object_tree.iter('bndbox'):
      xmin = float(bounding_box.find('xmin').text)
      ymin = float(bounding_box.find('ymin').text)
      xmax = float(bounding_box.find('xmax').text)
      ymax = float(bounding_box.find('ymax').text)
      break #WE TAKE ONLY ONE BOUNDING BOX INTO CONSIDERATION HERE
    class_name = object_tree.find('name').text
    class_dict={classes[i]:i for i in range(len(classes))} #FOR LABEL ENCODING CLASS(NOT ONE HOT ENCODING)
    #TO GET POSITION IN (x_cj, y_cj, w/width, h/height, label_encoded_class) NORMALIZED FORMAT
    bounding_box = [
        (xmin+xmax)/(2*width),(ymin+ymax)/(2*height),(xmax-xmin)/width,
        (ymax-ymin)/height,class_dict[class_name]]
    bounding_boxes.append(bounding_box)
  return tf.convert_to_tensor(bounding_boxes)

In [ ]:
preprocess_xml(val_maps+"2007_000032.xml")

In [ ]:
def generate_output(bounding_boxes):
  output_label = np.zeros((split_size, split_size, obj_ind))
  for c in range(len(bounding_boxes)): #THE NUMBER OF OBJECTS
    grid_x = bounding_boxes[...,c,0] * split_size #IN bounding_boxes[...,b,0] the ... IN INDEX IS TO ACCOUNT FOR BATCH
    grid_y = bounding_boxes[...,c,1] * split_size
    i = int(grid_x)
    j = int(grid_y)

    output_label[i,j,0:5] = [1., grid_x%1, grid_y%1, bounding_boxes[...,c,2], bounding_boxes[...,c,3]]
    output_label[i,j, 5 + bounding_boxes[...,c,4]] = 1. #TO ASSIGN ONE HOT ENCODED LABEL

  return tf.convert_to_tensor(output_label, tf.float32)

In [ ]:
generate_output(preprocess_xml(val_maps+"2007_000032.xml"))[3][0]

In [ ]:
generate_output(preprocess_xml(val_maps+"2007_000032.xml"))[3][3]

In [ ]:
def get_imbboxes(im_path,xml_path): #TO GET RESIZED IMAGES AND BOUNDING BOXES
  img=tf.io.decode_jpeg(tf.io.read_file(im_path))
  img=tf.cast(tf.image.resize(img, [h, w]),dtype=tf.float32)

  bboxes=tf.numpy_function(func=preprocess_xml, inp=[xml_path], Tout=tf.float32)
  return img,bboxes

In [ ]:
train_dataset = train_dataset.map(get_imbboxes)
val_dataset = val_dataset.map(get_imbboxes)

In [ ]:
for i,j in train_dataset.skip(2):
  print(i.shape,j)
  break

In [ ]:
cv2.imshow(np.array(i))

In [ ]:
def preprocess(img, bboxes):
  labels = tf.numpy_function(func = generate_output, inp = [bboxes], Tout = tf.float32)
  return img, labels

In [ ]:
for i,j in train_dataset.skip(2):
  print(i.shape,j)
  break

In [ ]:
transforms = A.Compose([
    A.Resize(h,w),
    A.RandomCrop(
         width=np.random.randint(int(0.9*w),w),
         height=np.random.randint(int(0.9*h),h), p=0.5),
    A.RandomScale(scale_limit=0.1, interpolation=cv2.INTER_LANCZOS4,p=0.5),
    A.HorizontalFlip(p=0.5,),
    A.Resize(h, w)
], #bbox_params=A.BboxParams(format='yolo', min_area = 25, min_visibility = 0.3))
bbox_params=A.BboxParams(format='yolo'))

In [ ]:
def aug_albument(image,bboxes):
  augmented=transforms(image=image,bboxes=bboxes)
  return [tf.convert_to_tensor(augmented["image"],dtype=tf.float32),
          tf.convert_to_tensor(augmented["bboxes"],dtype=tf.float32)]

In [ ]:
def process_data(image,bboxes):
    aug= tf.numpy_function(func=aug_albument, inp=[image,bboxes], Tout=(tf.float32,tf.float32))
    return aug[0],aug[1]

In [ ]:
train_dataset=train_dataset.map(process_data)

In [ ]:
for i,j in train_dataset.skip(2):
  print(i.shape,j)
  break

In [ ]:
cv2.imshow(np.array(i))

In [ ]:
def preprocess_augment(img,y):
  img = tf.image.random_brightness(img, max_delta=50.)
  img = tf.image.random_saturation(img, lower=0.5, upper=1.5)
  img = tf.image.random_contrast(img, lower=0.5, upper=1.5)
  #img = tf.image.random_hue(img, max_delta=0.5 )
  img = tf.clip_by_value(img, 0, 255)
  labels=tf.numpy_function(func=generate_output, inp=[y], Tout=(tf.float32))
  return img,labels

In [ ]:
train_dataset=train_dataset.map(preprocess_augment)
val_dataset=val_dataset.map(preprocess)

In [ ]:
train_dataset = train_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

# **MODEL**

In [ ]:
#base_model = tf.keras.applications.resnet50.ResNet50(
base_model=tf.keras.applications.efficientnet.EfficientNetB1(
    weights='imagenet',
    input_shape=(h, w, 3),
    include_top=False,
)
base_model.trainable=False

In [ ]:
model=tf.keras.Sequential([
  base_model,
  Conv2D(n_filters,(3,3), padding = 'same',kernel_initializer='he_normal',),
  BatchNormalization(),
  LeakyReLU(alpha=0.1),

  Conv2D(n_filters,(3,3),padding = 'same',kernel_initializer='he_normal',),
  BatchNormalization(),
  LeakyReLU(alpha=0.1),

  Conv2D(n_filters,(3,3),padding = 'same',kernel_initializer='he_normal',),
  BatchNormalization(),
  LeakyReLU(alpha=0.1),

  Conv2D(n_filters,(3,3),padding = 'same',kernel_initializer='he_normal',),
  LeakyReLU(alpha=0.1),

  Flatten(),

  Dense(n_filters,kernel_initializer='he_normal',),
  BatchNormalization(),
  LeakyReLU(alpha=0.1),

  Dropout(0.5),

  Dense(split_size * split_size * obj_ind,activation='sigmoid'),

  Reshape((split_size, split_size, obj_ind)),
])
model.summary()

# **COMPUTING LOSS**

In [ ]:
def compute_iou(boxes1, boxes2):
    boxes1_t = tf.stack([boxes1[..., 0] - boxes1[..., 2] / 2.0,
                         boxes1[..., 1] - boxes1[..., 3] / 2.0,
                         boxes1[..., 0] + boxes1[..., 2] / 2.0,
                         boxes1[..., 1] + boxes1[..., 3] / 2.0],
                        axis=-1)

    boxes2_t = tf.stack([boxes2[..., 0] - boxes2[..., 2] / 2.0,
                         boxes2[..., 1] - boxes2[..., 3] / 2.0,
                         boxes2[..., 0] + boxes2[..., 2] / 2.0,
                         boxes2[..., 1] + boxes2[..., 3] / 2.0],
                        axis=-1)
    lu = tf.maximum(boxes1_t[..., :2], boxes2_t[..., :2])
    rd = tf.minimum(boxes1_t[..., 2:], boxes2_t[..., 2:])

    intersection = tf.maximum(0.0, rd - lu)
    inter_square = intersection[..., 0] * intersection[..., 1]

    square1 = boxes1[..., 2] * boxes1[..., 3]
    square2 = boxes2[..., 2] * boxes2[..., 3]

    union_square = tf.maximum(square1 + square2 - inter_square, 1e-10)
    return tf.clip_by_value(inter_square / union_square, 0.0, 1.0)

In [ ]:
def difference(x,y):
  return tf.reduce_sum(tf.square(y-x))

In [ ]:
def yolo_loss(y_true, y_pred):
  target = y_true[...,0]

  ### OBJECT LOSS ###
  y_pred_extract = tf.gather_nd(y_pred, tf.where(target[:]==1)) #GATHERS Y_PRED WHERE THERE ARE OBJECTS
  y_target_extract = tf.gather_nd(y_true, tf.where(target[:]==1)) #GATHERS Y_TRUE WHERE THERE ARE OBJECTS

  rescaler = tf.where(target[:]==1) * split_size
  upscaler_1 = tf.concat([rescaler[:,1:],tf.zeros([len(rescaler),2], dtype=tf.int64)],axis=-1) #GIVES DISTANCE AND OTHER INFO(NOT NORMALIZED) UPTO THE GRID TOP LEFT CORNER

  #ALL THESE 3 VARIABLES GIVE DISTANCE AND OTHER INFO (NOT NORMALIZED) FROM GRID TOP LEFT CORNER TO GRID CENTER
  target_upscaler_2 = tf.repeat([[float(split_size),float(split_size), h, w]],
                       repeats=[len(rescaler)], axis=0)*tf.cast(y_target_extract[...,1:5], dtype = tf.float32) #WE USE TF.REPEAT IN CASE THERE ARE MULTIPLE ONJECTS
  pred_1_upscaler_2 = tf.repeat([[float(split_size),float(split_size), h, w]],
                      repeats=[len(rescaler)], axis=0)*tf.cast(y_pred_extract[...,1:5], dtype = tf.float32)
  pred_2_upscaler_2 = tf.repeat([[float(split_size),float(split_size), h, w]],
                      repeats=[len(rescaler)], axis=0)*tf.cast(y_pred_extract[...,6:10], dtype = tf.float32)

  target_orig = tf.cast(upscaler_1, dtype = tf.float32)+target_upscaler_2
  pred_1_orig = tf.cast(upscaler_1, dtype = tf.float32)+pred_1_upscaler_2
  pred_2_orig = tf.cast(upscaler_1, dtype = tf.float32)+pred_2_upscaler_2

  mask =tf.cast(tf.math.greater(compute_iou(target_orig,pred_2_orig),
                                compute_iou(target_orig,pred_1_orig)),dtype=tf.int32) #OUTPUT [0] (False TYPECASTED AS INT) IF compute_iou(target_orig,pred_1_orig) IS LARGER THAN compute_iou(target_orig,pred_2_orig) FOR A SINGLE OBJECT

  y_pred_joined=tf.transpose(tf.concat([tf.expand_dims(y_pred_extract[...,0],axis=0),
                        tf.expand_dims(y_pred_extract[...,5],axis=0)],axis=0)) #GETTING THE OBJECT PRESENT OR NOT PROBABILITY(SINCE SIGMOID) VALUES FOR BOTH BOUNDING BOXES

  obj_pred = tf.gather_nd(y_pred_joined,tf.stack([tf.range(len(rescaler)),mask],axis=-1)) #GATHER OBJECT PRESENT OR NOT PROBABILITY(SINCE SIGMOID) VALUE BASED ON THE MASK

  object_loss = difference(tf.cast(obj_pred,dtype =tf.float32)
                            ,tf.cast(tf.ones([len(rescaler)]),dtype=tf.float32)) #FINDING 1 - OBJ_PRED VALUE FOR EACH OBJECT

  ### NO OBJECT LOSS ###
  y_pred_extract = tf.gather_nd(y_pred[...,0:b*5], tf.where(target[:]==0))
  y_target_extract = tf.zeros(len(y_pred_extract))

  no_object_loss_1 = difference(tf.cast(y_pred_extract[...,0],dtype =tf.float32) ,tf.cast(y_target_extract,dtype=tf.float32))

  no_object_loss_2 = difference(tf.cast(y_pred_extract[...,5],dtype =tf.float32) ,tf.cast(y_target_extract,dtype=tf.float32))

  no_object_loss = no_object_loss_1+no_object_loss_2

  ### FOR OBJECT CLASSIFICATION LOSS ###
  y_pred_extract = tf.gather_nd(y_pred[...,10:],tf.where(target[:]==1))
  class_extract = tf.gather_nd(y_true[...,5:],tf.where(target[:]==1))

  class_loss = difference(tf.cast(y_pred_extract,dtype =tf.float32) ,tf.cast(class_extract,dtype=tf.float32))

  ### FOR OBJECT COORDINATES LOSS ###
  y_pred_extract = tf.gather_nd(y_pred[...,0:B*5], tf.where(target[:]==1))
  centre_joined=tf.stack([y_pred_extract[...,1:3],y_pred_extract[...,6:8]],axis=1) #GET THE CENTRE COORDINATES x_cj, y_cj FROM BOTH BOUNDING BOXES

  centre_pred = tf.gather_nd(centre_joined,tf.stack([tf.range(len(rescaler)),mask],axis=-1)) #TO GET THE CLOSEST BOUNDING BOX COORDINATES

  centre_target = tf.gather_nd(y_true[...,1:3], tf.where(target[:]==1)) #Y_TRUE CENTRE COORDINATES

  centre_loss = difference(centre_pred,centre_target)

  size_joined=tf.stack([y_pred_extract[...,3:5],y_pred_extract[...,8:10]],axis=1)

  size_pred = tf.gather_nd(size_joined,tf.stack([tf.range(len(rescaler)),mask],axis=-1))

  size_target = tf.gather_nd(y_true[...,3:5], tf.where(target[:]==1))

  size_loss = difference(tf.math.sqrt(tf.math.abs(size_pred)),tf.math.sqrt(tf.math.abs(size_target)))

  box_loss = centre_loss+size_loss

  ### LAMBDA VALUES ###
  lambda_coord = 5.0
  lambda_no_obj = 0.5

  ### TOTAL LOSS ###
  loss = object_loss + (lambda_no_obj*no_object_loss)+ tf.cast(lambda_coord*box_loss,dtype=tf.float32)+ tf.cast(class_loss,dtype=tf.float32)
  return loss

# **CALLBACKS AND MODEL TRAINING**

In [ ]:
checkpoint_filepath='/content/drive/MyDrive/Bang/yolo_efficientnet_B1_.h5'
callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True
)

In [ ]:
def scheduler(epoch, lr):
  if epoch < 40:
    return 1e-3
  elif epoch>=40 and epoch<80:
    return 5e-4
  else:
    return 1e-4

In [ ]:
lr_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [ ]:
model.compile( loss=yolo_loss, optimizer=Adam(1e-3))

In [ ]:
#model.load_weights(checkpoint_filepath)

In [ ]:
history = model.fit(
  train_dataset,
  validation_data=val_dataset,
  verbose=1,
  epochs=135,
  callbacks = [lr_callback,callback]
)

# **TESTING**

In [ ]:
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d awsaf49/coco-2017-dataset

In [ ]:
!unzip "/content/pascal-voc-2012.zip" -d "/content/TESTdataset"

In [ ]:
model.load_weights(checkpoint_filepath)

In [ ]:
!mkdir outputs/

In [ ]:
coco_path = '/content/TESTdataset/coco_images/'

In [ ]:
def model_test(filename):
  try:
    test_path = coco_path + filename

    img=cv2.resize(cv2.imread(test_path),(h, w))

    image=tf.io.decode_jpeg(tf.io.read_file(test_path))
    image=tf.image.resize(image, [h, w])

    output=model.predict(np.expand_dims(image, axis = 0))

    threshold = 0.25

    object_positions=tf.concat([tf.where(output[...,0] >= threshold),tf.where(output[...,5] >= threshold)],axis=0)
    #print(object_positions)
    selected_output=tf.gather_nd(output,object_positions)
    #print(selected_output)

    final_boxes=[]
    final_scores=[]

    for i, pos in enumerate(object_positions):
      for j in range(2): # 2 DENOTES NO. OF BOUNDING BLOCKS
        if selected_output[i][j*5] > threshold:
          output_box=tf.cast(output[pos[0]][pos[1]][pos[2]][(j*5)+1:(j*5)+5],dtype=tf.float32) # SELECT A BOUNDING BOX COORDINATES (4 NUMBERS) FOR THE SPECIFIC GRID CELL CONTAINING OBJECT

          # TO CONVERT OUTPUT IN X_CJ, Y_CJ, WIDTH(NORMALIZED), HEIGHT(NORMALIZED) TO X_MIN, Y_MIN, X_MAX, Y_MAX, CLASS_NAME FORMAT
          x_centre=(tf.cast(pos[1],dtype=tf.float32)+output_box[0])*32
          y_centre=(tf.cast(pos[2],dtype=tf.float32)+output_box[1])*32

          x_width,y_height=tf.math.abs(h * output_box[2]),tf.math.abs(w * output_box[3])

          x_min,y_min=int(x_centre-(x_width/2)),int(y_centre-(y_height/2))
          x_max,y_max=int(x_centre+(x_width/2)),int(y_centre+(y_height/2))

          if(x_min<=0):x_min = 0
          if(y_min<=0):y_min = 0
          if(x_max>=W):x_max = w
          if(y_max>=H):y_max = h

          final_boxes.append([x_min,y_min,x_max,y_max,
              str(classes[tf.argmax(selected_output[...,10:],axis=-1)[i]])]) # GIVES CLASS NAME

          final_scores.append(selected_output[i][j*5]) #APPENDS OBJECT PRESENT OR NOT PREDICTION
    print(final_scores)
    print('finalboxes',final_boxes)

    final_boxes=np.array(final_boxes)

    # NON MAX SUPRESSION
    object_classes=final_boxes[...,4]
    nms_boxes=final_boxes[...,0:4] #GETTING IN X_MIN, Y_MIN, X_MAX, Y_MAX FORMAT

    nms_output=tf.image.non_max_suppression(
        nms_boxes,final_scores,
        max_output_size=100,
        iou_threshold=0.2,
        score_threshold=float('-inf')
    )

    #VISUALIZING OUTPUT
    for i in nms_output:
      cv2.rectangle(img,
          (int(final_boxes[i][0]),int(final_boxes[i][1])),
          (int(final_boxes[i][2]),int(final_boxes[i][3])),
           (0,0,255),1)  #(0,0,255) REPRESENTS COLOR OF RECTANGLE
      cv2.putText(img,
          final_boxes[i][-1],
          (int(final_boxes[i][0]),int(final_boxes[i][1])+15),
          cv2.FONT_HERSHEY_COMPLEX_SMALL,1,(2,225,155),1)
    cv2.imwrite('/content/outputs/'+filename[:-4]+'_det'+'.jpg',cv2.resize(img,(384,384)))
  except:
    print("no object found!")

In [ ]:
for filename in os.listdir(coco_path):
  model_test(filename)